In [27]:
import os
os.environ['http_proxy'] = "127.0.0.1:7890"
os.environ['https_proxy'] = "127.0.0.1:7890"

In [28]:
import requests
requests.get("https://google.com")

<Response [200]>

In [29]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [30]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [31]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [33]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [34]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss} [{current} / {size}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(dim=1) == y).float().sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [35]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n---------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------
loss: 2.3071253299713135 [64 / 60000]
loss: 2.3010106086730957 [6464 / 60000]
loss: 2.279904365539551 [12864 / 60000]
loss: 2.2726480960845947 [19264 / 60000]
loss: 2.2596771717071533 [25664 / 60000]
loss: 2.2193024158477783 [32064 / 60000]
loss: 2.227667808532715 [38464 / 60000]
loss: 2.2089638710021973 [44864 / 60000]
loss: 2.1915502548217773 [51264 / 60000]
loss: 2.1541576385498047 [57664 / 60000]
Test Error: 
 Accuracy: 51.5%, Avg loss: 2.157662 

Epoch 2
---------
loss: 2.135371208190918 [64 / 60000]
loss: 2.150287389755249 [6464 / 60000]
loss: 2.0917327404022217 [12864 / 60000]
loss: 2.051964521408081 [19264 / 60000]
loss: 2.044734477996826 [25664 / 60000]
loss: 2.004573345184326 [32064 / 60000]
loss: 2.024864912033081 [38464 / 60000]
loss: 1.9254121780395508 [44864 / 60000]
loss: 1.9395885467529297 [51264 / 60000]
loss: 1.8747938871383667 [57664 / 60000]
Test Error: 
 Accuracy: 58.7%, Avg loss: 1.875840 

Epoch 3
---------
loss: 1.8518821001052856 [64 / 60000]


In [37]:
torch.save(model.state_dict(), "models/start.pth")
print("saved")

saved


In [40]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("models/start.pth"))

<All keys matched successfully>

In [41]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
